This is all using GDAS. I don't think I want that- I think I want regular GFS (not positive.)

In [ ]:

import fsspec
import xarray as xr

In [ ]:
# 1. Point s3fs at the public bucket (no credentials needed)
fs = fsspec.fileSystem("s3", anon=True)
date = 20250624
# 2. List a directory (e.g. daily time_series files for soil moisture)
path = f"noaa-gfs-bdp-pds/gdas.{date}/00/atmos/"  # adjust to your year/month dynamically
file = fs.glob(
    path + "gdas.t00z.sfcanl.nc"
) 

with fs.open(file) as f:
    ds = xr.open_dataset(f, engine="netcdf4")


In [ ]:
gfs = xr.open_dataset(file, engine="netcdf4")

In [ ]:
grib_file = "data/gfs/gdas.t00z.sfluxgrbf000.grib2"
# 3. Open as a single concatenated Dataset
ds_surface = xr.open_dataset(
    grib_file,
    engine="cfgrib",
    filter_by_keys={'typeOfLevel': 'surface'}, #can also add decode_coords="all" for compatability with rioxarray if needed. 
)

grib_file = "data/gfs/gdas.t00z.sfluxgrbf000.grib2"
# 3. Open as a single concatenated Dataset
ds_below = xr.open_dataset(
    grib_file,
    engine="cfgrib",
    filter_by_keys={'typeOfLevel': 'depthBelowLandLayer', 'level': 0.0},
)

In [ ]:
ds_surface

In [ ]:
ds_surface_us = ds_surface.sel(longitude=slice((180-126.59)+180,(180-65.6)+180), latitude=slice(50.59, 24.4))
ds_surface_us

In [ ]:
ds_surface_us[['wilt','fldcp','SUNSD', 'sltyp']]

In [ ]:
ds_below

In [ ]:
ds_below_us = ds_below.sel(longitude=slice((180-126.59)+180,(180-65.6)+180), latitude=slice(50.59, 24.4))

In [ ]:
us_soilw = ds_below_us['soilw']

In [ ]:
us_soilw_masked = us_soilw.where(us_soilw != us_soilw.attrs['GRIB_missingValue']) # Remove the missing values (like ocean, etc) so plots are better

In [ ]:
us_soilw_masked

In [ ]:
us_soilw_surface_masked = us_soilw_masked.sel(depthBelowLandLayer='0.')

In [ ]:
us_soilw_surface_masked.plot()

Would I want to use something like ds.time.dt.floor("D") to resample everything back to the zero hour of the day? I guess this is a good way to aggregate data from different hours into one day. Maybe if I pull two 6 hourly forecasts per day?